# 미세먼지 예측 향상

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import glob

## 1. 데이터 불러오기

In [46]:
data1 = pd.read_csv("data/OA00001.csv", encoding='euc-kr')
data2 = pd.read_csv("data/OA00003.csv", encoding='euc-kr')
data3 = pd.read_csv("data/OA00004.csv", encoding='euc-kr')
data4 = pd.read_csv("data/OA00005.csv", encoding='euc-kr')
data5 = pd.read_csv("data/OA00007.csv", encoding='euc-kr')
data6 = pd.read_csv("data/OA00008.csv", encoding='euc-kr')
data7 = pd.read_csv("data/OA00010.csv", encoding='euc-kr')
target = pd.read_excel("data/보건환경연구원.xlsx")

## 2. 데이터 전처리

In [7]:
target

,Date,pm25,pm10,so2,no2
0,2021-10-29 15:00:00,1.0,10.0,0.003,0.007
1,2021-10-29 14:00:00,2.0,11.0,0.003,0.007
2,2021-10-29 13:00:00,3.0,16.0,0.003,0.006
3,2021-10-29 12:00:00,1.0,15.0,0.003,0.008
4,2021-10-29 11:00:00,7.0,15.0,0.003,0.009
...,...,...,...,...,...
567,2021-10-06 00:00:00,17.0,20.0,0.002,0.020
568,2021-10-05 23:00:00,13.0,24.0,0.002,0.018
569,2021-10-05 22:00:00,17.0,28.0,0.002,0.021
570,2021-10-05 21:00:00,11.0,25.0,0.003,0.015


In [3]:
data1

,reg_date,device_id,pm25,pm10,so2,no2,temp,humi
0,2021-10-05 20:00,OA00001,5.75,5.76,0.004,0.027,26.3,61.5
1,2021-10-05 20:01,OA00001,6.06,6.06,0.004,0.027,26.2,61.7
2,2021-10-05 20:02,OA00001,5.84,5.84,0.004,0.027,26.2,61.8
3,2021-10-05 20:03,OA00001,5.32,5.32,0.004,0.027,26.2,61.9
4,2021-10-05 20:04,OA00001,5.95,5.95,0.004,0.026,26.1,62.2
...,...,...,...,...,...,...,...,...
34009,2021-10-29 14:55,OA00001,2.31,2.31,0.004,0.017,24.5,29.8
34010,2021-10-29 14:56,OA00001,2.07,2.07,0.004,0.018,24.4,30.1
34011,2021-10-29 14:57,OA00001,1.99,1.99,0.004,0.017,24.5,30.0
34012,2021-10-29 14:58,OA00001,2.10,2.10,0.004,0.019,24.5,29.7


In [47]:
data1['Date'] = pd.to_datetime(data1['reg_date'])
data1['device_id'] = 1
data1.set_index('Date', inplace=True)
data1 = data1.resample('1H').mean()
data1.reset_index(inplace=True)

data2['Date'] = pd.to_datetime(data2['reg_date'])
data2['device_id'] = 2
data2.set_index('Date', inplace=True)
data2 = data2.resample('1H').mean()
data2.reset_index(inplace=True)

data3['Date'] = pd.to_datetime(data3['reg_date'])
data3['device_id'] = 3
data3.set_index('Date', inplace=True)
data3 = data3.resample('1H').mean()
data3.reset_index(inplace=True)

data4['Date'] = pd.to_datetime(data4['reg_date'])
data4['device_id'] = 4
data4.set_index('Date', inplace=True)
data4 = data4.resample('1H').mean()
data4.reset_index(inplace=True)

data5['Date'] = pd.to_datetime(data5['reg_date'])
data5['device_id'] = 5
data5.set_index('Date', inplace=True)
data5 = data5.resample('1H').mean()
data5.reset_index(inplace=True)

data6['Date'] = pd.to_datetime(data6['reg_date'])
data6['device_id'] = 6
data6.set_index('Date', inplace=True)
data6 = data6.resample('1H').mean()
data6.reset_index(inplace=True)

data7['Date'] = pd.to_datetime(data7['reg_date'])
data7['device_id'] = 7
data7.set_index('Date', inplace=True)
data7 = data7.resample('1H').mean()
data7.reset_index(inplace=True)

In [49]:
data7

,Date,device_id,pm25,pm10,so2,no2,temp,humi
0,2021-10-05 20:00:00,7.0,9.754727,9.772364,0.007000,0.067855,25.056909,66.141273
1,2021-10-05 21:00:00,7.0,10.613273,10.635636,0.007000,0.067491,24.321273,69.403273
2,2021-10-05 22:00:00,7.0,12.001034,12.056897,0.007000,0.070586,23.751552,71.658103
3,2021-10-05 23:00:00,7.0,12.021852,12.072778,0.007000,0.055593,22.950741,74.519259
4,2021-10-06 00:00:00,7.0,12.497544,12.524386,0.006491,0.051140,22.419123,76.681053
...,...,...,...,...,...,...,...,...
566,2021-10-29 10:00:00,7.0,3.670526,3.678772,0.005000,0.022491,17.526316,47.549123
567,2021-10-29 11:00:00,7.0,3.745667,3.753000,0.005000,0.020300,18.810667,43.137000
568,2021-10-29 12:00:00,7.0,3.272931,3.277241,0.005000,0.019672,19.769483,39.559828
569,2021-10-29 13:00:00,7.0,2.852105,2.855439,0.005000,0.019193,20.742281,35.128421


In [50]:
df_data1 = pd.merge(data1, target, on='Date')
df_data2 = pd.merge(data2, target, on='Date')
df_data3 = pd.merge(data3, target, on='Date')
df_data4 = pd.merge(data4, target, on='Date')
df_data5 = pd.merge(data5, target, on='Date')
df_data6 = pd.merge(data6, target, on='Date')
df_data7 = pd.merge(data7, target, on='Date')

In [54]:
df_data4

,Date,device_id,pm25_x,pm10_x,so2_x,no2_x,temp,humi,pm25_y,pm10_y,so2_y,no2_y
0,2021-10-05 20:00:00,4.0,11.154000,11.173500,0.006983,0.053867,27.231500,57.844333,12.0,24.0,0.003,0.012
1,2021-10-05 21:00:00,4.0,11.357333,11.374833,0.006983,0.054217,26.571000,61.008667,11.0,25.0,0.003,0.015
2,2021-10-05 22:00:00,4.0,9.969667,10.005333,0.006433,0.049900,26.053500,63.258167,17.0,28.0,0.002,0.021
3,2021-10-05 23:00:00,4.0,10.887500,10.948833,0.006133,0.044100,25.419500,65.304000,13.0,24.0,0.002,0.018
4,2021-10-06 00:00:00,4.0,11.563667,11.619167,0.006000,0.043783,24.556000,67.837667,17.0,20.0,0.002,0.020
...,...,...,...,...,...,...,...,...,...,...,...,...
566,2021-10-29 10:00:00,4.0,3.918305,3.922712,0.005000,0.033271,17.358644,46.840339,11.0,22.0,0.003,0.016
567,2021-10-29 11:00:00,4.0,3.489167,3.499500,0.005000,0.026183,18.536000,42.427000,7.0,15.0,0.003,0.009
568,2021-10-29 12:00:00,4.0,3.021833,3.024000,0.005000,0.021050,19.589167,37.168000,1.0,15.0,0.003,0.008
569,2021-10-29 13:00:00,4.0,2.745833,2.748500,0.005000,0.020050,20.817000,33.673167,3.0,16.0,0.003,0.006


In [74]:
df_data = pd.concat([df_data1, df_data2, df_data3, df_data4, df_data5, df_data6, df_data7])

In [75]:
df_data.set_index(df_data['Date'], inplace=True)
df_data.drop(['Date'], axis=1, inplace=True)
df_data.dropna(inplace=True)
df_data

,device_id,pm25_x,pm10_x,so2_x,no2_x,temp,humi,pm25_y,pm10_y,so2_y,no2_y
Date,,,,,,,,,,,
2021-10-05 20:00:00,1.0,6.503000,6.504667,0.004000,0.026533,25.626667,64.223333,12.0,24.0,0.003,0.012
2021-10-05 21:00:00,1.0,7.305333,7.307833,0.004000,0.027033,24.781667,68.271667,11.0,25.0,0.003,0.015
2021-10-05 22:00:00,1.0,8.621500,9.860167,0.004000,0.027150,24.231667,70.891667,17.0,28.0,0.002,0.021
2021-10-05 23:00:00,1.0,8.311500,8.323167,0.004000,0.026133,23.463333,73.615000,13.0,24.0,0.002,0.018
2021-10-06 00:00:00,1.0,8.686000,8.705333,0.004000,0.025683,22.983333,74.820000,17.0,20.0,0.002,0.020
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-29 10:00:00,7.0,3.670526,3.678772,0.005000,0.022491,17.526316,47.549123,11.0,22.0,0.003,0.016
2021-10-29 11:00:00,7.0,3.745667,3.753000,0.005000,0.020300,18.810667,43.137000,7.0,15.0,0.003,0.009
2021-10-29 12:00:00,7.0,3.272931,3.277241,0.005000,0.019672,19.769483,39.559828,1.0,15.0,0.003,0.008


## 3. 모델
### 3-1. 선형회귀

In [77]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# X, y 분리
X = df_data.drop(['pm25_y'], axis=1)
y = df_data['pm25_y']

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터분리
X_train , X_test, y_train , y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)

# 모델
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [78]:
def print_metrics(y, y_pred, average = 'micro', title=None):
    mse = np.round(mean_squared_error(y, y_pred), 3)
    rmse = np.round(np.sqrt(mse), 3)
    mae = np.round(mean_absolute_error(y, y_pred), 3)
    r2 = np.round(r2_score(y, y_pred), 3)
    if title:
        print(title)
    print(f"MSE:{mse}, RMSE:{rmse},MAE:{mae}, R2:{r2}")

In [79]:
# 예측
y_pred = lr.predict(X_test)
print_metrics(y_test, y_pred, title='LR')

LR
MSE:15.651, RMSE:3.956,MAE:3.031, R2:0.795


### 3-2. XGB

In [81]:
from xgboost import XGBRegressor

# X, y 분리
X = df_data.drop(['pm25_y'], axis=1)
y = df_data['pm25_y']

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터분리
X_train , X_test, y_train , y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)

# 모델
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

# 예측
y_pred = xgb.predict(X_test)
print_metrics(y_test, y_pred, title='XGBRegressor')

XGBRegressor
MSE:5.407, RMSE:2.325,MAE:1.728, R2:0.929


## 4. 다른 데이터세트

In [91]:
df_another = df_data.copy()
df_another.drop(['pm10_y', 'so2_y', 'no2_y', 'device_id'], axis=1, inplace=True)
df_another

,pm25_x,pm10_x,so2_x,no2_x,temp,humi,pm25_y
Date,,,,,,,
2021-10-05 20:00:00,6.503000,6.504667,0.004000,0.026533,25.626667,64.223333,12.0
2021-10-05 21:00:00,7.305333,7.307833,0.004000,0.027033,24.781667,68.271667,11.0
2021-10-05 22:00:00,8.621500,9.860167,0.004000,0.027150,24.231667,70.891667,17.0
2021-10-05 23:00:00,8.311500,8.323167,0.004000,0.026133,23.463333,73.615000,13.0
2021-10-06 00:00:00,8.686000,8.705333,0.004000,0.025683,22.983333,74.820000,17.0
...,...,...,...,...,...,...,...
2021-10-29 10:00:00,3.670526,3.678772,0.005000,0.022491,17.526316,47.549123,11.0
2021-10-29 11:00:00,3.745667,3.753000,0.005000,0.020300,18.810667,43.137000,7.0
2021-10-29 12:00:00,3.272931,3.277241,0.005000,0.019672,19.769483,39.559828,1.0


In [92]:
# 선형회귀
# X, y 분리
X = df_another.drop(['pm25_y'], axis=1)
y = df_another['pm25_y']

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터분리
X_train , X_test, y_train , y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)

# 모델
lr = LinearRegression()
lr.fit(X_train, y_train)

# 예측
y_pred = lr.predict(X_test)
print_metrics(y_test, y_pred, title='LR')

LR
MSE:30.126, RMSE:5.489,MAE:4.049, R2:0.605


In [93]:
# X, y 분리
X = df_another.drop(['pm25_y'], axis=1)
y = df_another['pm25_y']

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터분리
X_train , X_test, y_train , y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)

# 모델
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

# 예측
y_pred = xgb.predict(X_test)
print_metrics(y_test, y_pred, title='XGBRegressor')

XGBRegressor
MSE:23.543, RMSE:4.852,MAE:3.531, R2:0.691
